In [9]:
import xarray as xr
import matplotlib.pyplot as plt
import os
import numpy as np
# import metview as mv

In [ ]:
basePath = "/mnt/qb/goswami/data/era5"
# path = os.path.join(basePath, 'single_pressure_level', sfc)
print("loading data")
ds = xr.open_mfdataset(os.path.join(basePath, 'single_pressure_level', '10m_v_component_of_wind', "10m_v_component_of_wind_????.nc"),parallel=True)
print("ds: ")
print(ds.info())
print("fin loading")

In [ ]:
basePath_sfno = "/mnt/qb/work2/goswami0/gkd965/outputs/sfno"
file = "v10_SFNO_HalfYearForecast_startfrom_2022-01-01.nc"
# path = os.path.join(basePath, 'single_pressure_level', sfc)
print("loading sfno data")
ds_sfno = xr.open_dataset(os.path.join(basePath_sfno,file), chunks={"step": 4})
print("ds sfno: ")
print(ds_sfno.info())
print("fin loading")

In [ ]:
Model = ds.drop([np.datetime64('02-29')], dim='time')

In [ ]:
ds['hourofyear'] = xr.DataArray(ds.time.dt.strftime('%m-%d %H'), coords=ds.time.coords)
ds_hourofyear = ds.groupby("hourofyear")
mean_temps = ds_hourofyear.mean()

In [ ]:
Ref = mean_temps.to_array(dim='2m_temperature')
Model = ds_sfno.to_array(dim='2t')

In [ ]:
RMSE = 1 - Model / Ref